In [ ]:
#Importing the libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pybaseball as pyb
from pybaseball import statcast
import xgboost as xgb
import numpy as np
from scipy.stats import rankdata 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.lines 
from matplotlib.lines import Line2D
from scipy.stats import gaussian_kde

In [ ]:
# create a heatmap visualation of each pitch and the strike zone 
df = pd.read_csv("/Users/johndavis/Desktop/merged.csv",low_memory=False)

In [ ]:
#Create a new dataframe with only the columns we want for Analysis
df = df[['Pitcher','RelSpeed','AutoPitchType','PitchCall','SpinRate','RelHeight','RelSide','Extension','InducedVertBreak', 'HorzBreak','PlateLocHeight','PlateLocSide']]
#Rename the columns to be more descriptive
df.rename(columns={'RelSpeed':'Velocity', 'SpinRate':'Spin Rate', 'RelHeight':'Release Height', 'RelSide':'Release Side', 'Extension':'Extension', 'InducedVertBreak':'Induced Vertical Break', 'HorzBreak':'Horizontal Break'}, inplace=True)


In [ ]:
#add pitch result column
df.insert(3, 'Pitch Result', df['PitchCall'])

In [ ]:
#delete pitch call column
del df['PitchCall']

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
#change AutoPitchType to Pitch Type
df.rename(columns={'AutoPitchType':'Pitch Type'}, inplace=True)

In [ ]:
df.head()

In [ ]:
#create a fuunction that asks for a pitchers name then converts its from first last to last, first
def get_pitcher_name():
    first_name = input("Enter the pitchers first name: ")
    last_name = input("Enter the pitchers last name: ")
    pitcher_name = last_name + ", " + first_name
    return pitcher_name

In [ ]:
get_pitcher_name()

In [ ]:
#use get_pitcher_name so that the user can input the pitchers name and then use that name to create a heatmap
def get_pitch_heatmap(df, Pitcher, pitch_type):
    
    if pitch_type == "Breaking Ball":
        pitch_df = df.loc[(df['Pitcher'] == Pitcher) & (df['Pitch Type'].isin(["Slider", "Curveball"]))]
    elif pitch_type == "Fastball":
        pitch_df = df.loc[(df['Pitcher'] == Pitcher) & (df['Pitch Type'].isin(["Four-Seam", "Sinker", "Cutter"]))]
    elif pitch_type == "Offspeed":
        pitch_df = df.loc[(df['Pitcher'] == Pitcher) & (df['Pitch Type'].isin(["Changeup", "Splitter"]))]
    x = pitch_df['PlateLocSide']
    y = pitch_df['PlateLocHeight']
    k = gaussian_kde(np.vstack([x, y]))
    xi, yi = np.mgrid[-2:2:100j, 0:5:100j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_xlim([-2, 2])
    ax.set_ylim([0, 5])
    ax.set_xlabel("Horizontal Pitch Location")
    ax.set_ylabel("Vertical Pitch Location")
    #pitcher 2 is first name then last name instead of last name, first name
    Pitcher_2 = Pitcher.split(", ")
    Pitcher_2 = Pitcher_2[1] + " " + Pitcher_2[0]
    ax.set_title( Pitcher_2 + " " +pitch_type + " Pitch Heatmap - Pitcher's View")
    heatmap = ax.pcolormesh(xi, yi, zi.reshape(xi.shape), cmap=plt.cm.inferno_r, shading='auto')
    cbar = fig.colorbar(heatmap, ax=ax)
    cbar.ax.set_ylabel('Density')
    ax.add_patch(plt.Rectangle((-0.85, 1.6), 1.7, 1.8, fill=False, edgecolor='white', lw=2))
    #ax.add_patch(plt.Rectangle((-0.85, 1.6), 1.7, 1.8, fill=False, edgecolor='white', lw=2))
    plt.show()
    

In [ ]:
pitcher_name = get_pitcher_name()

In [ ]:
#create a function that asks for pitchers first and last name and outputs three heatmaps for breaking balls, fastballs, and offspeed pitches
def get_all_pitch_heatmaps(df, Pitcher):
    get_pitch_heatmap(df, Pitcher, "Fastball")
    get_pitch_heatmap(df, Pitcher, "Breaking Ball")
    get_pitch_heatmap(df, Pitcher, "Offspeed")

In [ ]:
get_all_pitch_heatmaps(df, pitcher_name)